In [1]:
pip install instagramy

  Using cached instagramy-4.5-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install instagramy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
from instagramy import InstagramUser
from instagramy import InstagramPost
import pandas as pd
import numpy as np
from scipy import stats
from scipy import special
import seaborn as sns
import matplotlib.pyplot as plt
session_id = "49871308460%3ArXSb3fTaS1uPzK%3A21"

In [4]:
#List of NBA Player Handles (Easy to change if a better list is available)
#player_handles = ['kingjames', 'stephencurry30', 'easymoneysniper', 'ygtrece', 'cp3', 'zo', 'giannis_an34', 'klaythompson', 'carmeloanthony', 'jimmybutler', 'traeyoung', 'dloading', 'jaytatum0', 'blakegriffin23', 'isaiahthomas', 'money23green', 'melo', 'demar_derozan', 'lukadoncic', 'nolimitherro', 'deandre', 'javalemcgee', '_kw15', 'acfresh21', 'vicoladipo', 'jmglitxh27', 'fchwpo', 'louwillville', 'hbarnes', 'qcook323', 'antdavis23', 'zionwilliamson', 'damianlillard', 'kuz', 'jharden13', 'russwest44', 'kyrieirving', 'joelembiid', 'johnwall']
player_handles = ['kingjames', au

In [5]:
#Getting the names of every player that is associatiod with the handle
player_names = []
for i in np.arange(0, len(player_handles)):
    player = InstagramUser(player_handles[i], sessionid=session_id)
    player_names.append(player.fullname)

In [6]:
#Cleaning the names of the player
player_names[2] = 'Kevin Durant'
player_names[len(player_names) - 3] = 'Kyrie Irving'
player_names[12] = 'Jayson Tatum'
player_names[20] = 'Deandre Jordan'
player_names[21] = 'Javale Mcgee'
player_names[len(player_names) - 2] = "Joel Embiid"
player_names[6] = 'Giannis Antetokounmpo'

In [7]:
#Function to find the number of likes per post
def avg_like_finder(player):
    i = 0
    like_total = 0
    while i < len(player.posts):
        like_total = like_total + player.posts[i][0]
        i = i + 1
    return like_total/len(player.posts)

In [8]:
#Function to find the number of comments per post
def avg_comments_finder(player):
    i = 0
    comment_total = 0
    while i < len(player.posts):
        comment_total = comment_total + player.posts[i][1]
        i = i + 1
    return comment_total/len(player.posts)

In [ ]:
#List of average likes each handle receives per post
avg_likes = []
for i in np.arange(0, len(player_handles)):
    player = InstagramUser(player_handles[i], sessionid=session_id)
    avg_likes.append(avg_like_finder(player))
    

In [ ]:
#List of average comments each handle receives per post
avg_comments = []
for i in np.arange(0, len(player_handles)):
    player = InstagramUser(player_handles[i], sessionid=session_id)
    avg_comments.append(avg_comments_finder(player))

In [ ]:
#List of number of followers each handle has
follower_count = []
for i in np.arange(0, len(player_handles)):
    player = InstagramUser(player_handles[i], sessionid=session_id)
    follower_count.append(player.number_of_followers)

In [ ]:
#Table with above lists
table = {'Player Name': player_names,
         'Instagram Handle': player_handles,
         'Likes Per Post': avg_likes,
         'Comments Per Post': avg_comments,
         'Number of Followers': follower_count,
        }

nba_inst = pd.DataFrame(table)

In [ ]:
#added engagement rate to the table ((avg likes + avg comments)/num of followers*100)
nba_inst['Engagement Rate'] = 100*(nba_inst['Likes Per Post'] + nba_inst['Comments Per Post'])/nba_inst['Number of Followers']
nba_inst.head(10)

In [ ]:
nba_inst.to_csv('WithoutRanks.csv', index = True)

In [ ]:
#Finding the ranks of each nba player in each category (likes, comments, followers, and engagement rate)
ranked_by_likes = len(nba_inst['Likes Per Post']) - stats.rankdata(nba_inst['Likes Per Post']).astype(int) + 1
ranked_by_comments = len(nba_inst['Comments Per Post']) - stats.rankdata(nba_inst['Comments Per Post']).astype(int) + 1
ranked_by_followers = len(nba_inst['Number of Followers']) - stats.rankdata(nba_inst['Number of Followers']).astype(int) + 1
ranked_by_engagement = len(nba_inst['Engagement Rate']) - stats.rankdata(nba_inst['Engagement Rate']).astype(int) + 1

#Table of these ranks
rank_table = {'Player Name': player_names,
              'Likes Rank': ranked_by_likes,
              'Comments Rank': ranked_by_comments,
              'Followers Rank': ranked_by_followers,
              'Engagement Rank': ranked_by_engagement
             }

nba_ranks = pd.DataFrame(rank_table)

#Taking the average of all four ranks for each player
avg_rank = []
for i in np.arange(0, len(nba_ranks['Player Name'])):
    L = nba_ranks['Likes Rank'][i]
    C = nba_ranks['Comments Rank'][i]
    F = nba_ranks['Followers Rank'][i]
    E = nba_ranks['Engagement Rank'][i]
    summed = L + C + F + E
    averaged = summed/4
    avg_rank.append(averaged)
    

#Adding the average rank to the last column of the rank table
nba_ranks['Average Rank'] = avg_rank
nba_ranks.head(10)

In [ ]:
#Sorted table by average rank
sorted_by_avg_rank = nba_ranks.sort_values('Average Rank', ascending = True)
sorted_by_avg_rank.head(10)

In [ ]:
#Merged the two data frames
all_data = nba_inst.merge(nba_ranks, how = 'inner', on = 'Player Name')

In [ ]:
#Reordered the columns so it is Likes, Likes Rank, Comments, Comments Rank, ... etc.
columns = all_data.columns.tolist()
columns = ['Player Name',
           'Instagram Handle',
           'Likes Per Post',
           'Likes Rank',
           'Comments Per Post',
           'Comments Rank',
           'Number of Followers',
           'Followers Rank',
           'Engagement Rate',
           'Engagement Rank',
           'Average Rank'
           ]

all_data = all_data[columns]
all_data.head(10)

In [ ]:
#Final Table Sorted by average rank
all_data_by_avg_rank = all_data.sort_values('Average Rank', ascending = True)
all_data_by_avg_rank.head(10)